# Auto-Sklearn

auto-sklearn is an automated machine learning toolkit and a drop-in replacement for a scikit-learn estimator.
It frees a machine learning user from algorithm selection and hyperparameter tuning. It leverages recent advantages in Bayesian optimization, meta-learning and ensemble construction.

[Docu](https://automl.github.io/auto-sklearn/master/),
[Paper](https://papers.nips.cc/paper/5872-efficient-and-robust-automated-machine-learning)

Notebook author: Marius Lindauer ([www.automl.org](https://www.automl.org))

# Installation

The notebook was created based on
scikit-learn 0.19.2, smac 0.8.0 and auto-sklearn 0.5.1.

In [ ]:
!apt-get install swig -y
!pip install Cython numpy

# sometimes you have to run the next command twice on colab
# I haven't figured out why
!pip install auto-sklearn

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (3.0.12-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# ignore some annoying warnings for demonstrating auto-sklearn
# shouldn't be done in real production
import numpy as np
np.warnings.filterwarnings('ignore')

# 1st Example

We want to train a classifier for the [digits](http://archive.ics.uci.edu/ml/datasets/Pen-Based+Recognition+of+Handwritten+Digits) dataset.
All we do is to split the dataset into training and test data,
pass the training data to auto-sklearn
and evaluate the trained model on test data.


In [ ]:
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

# Load data
X, y = sklearn.datasets.load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = \
        sklearn.model_selection.train_test_split(X, y, random_state=1)

In [ ]:
import autosklearn.classification

# configure auto-sklearn
automl = autosklearn.classification.AutoSklearnClassifier(
          time_left_for_this_task=120, # run auto-sklearn for at most 2min
          per_run_time_limit=30, # spend at most 30 sec for each model training
          )

# train model(s)
automl.fit(X_train, y_train)

# evaluate
y_hat = automl.predict(X_test)
test_acc = sklearn.metrics.accuracy_score(y_test, y_hat)
print("Test Accuracy score {0}".format(test_acc))

Test Accuracy score 0.9844444444444445


The accuracy might not be quite state-of-the-art after running auto-sklearn only for two minutes. If you want to achieve better results, please try to increase the time limit `time_left_for_this_task`.

## Inspect some statistics of our first example

Please note that auto-sklearn has internally used a holdout set of the traning set to estimate the quality of the trained model. Based on this hold-out validation set, auto-sklearn reports a validation score.

In [ ]:
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: 6fed480e-8068-11ed-807e-0242ac1c000c
  Metric: accuracy
  Best validation score: 0.988764
  Number of target algorithm runs: 14
  Number of successful target algorithm runs: 11
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 1



## Inspect Ensemble

Auto-sklearn considers all trained models as potential candidates to build an ensemble out of these.
The following command allows you to see the ensemble.

Since the ensemble does not use a simple majority voting, but a weighted ensemble,
the fomat is the following:

`(ensemble weight, machine learning pipeline)`

In [ ]:
print(automl.show_models())

{2: {'model_id': 2, 'rank': 1, 'cost': 0.024719101123595544, 'ensemble_weight': 0.16, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f55d59e5e80>, 'balancing': Balancing(random_state=1), 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f55d594a1c0>, 'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7f55d594a220>, 'sklearn_classifier': RandomForestClassifier(max_features=7, n_estimators=512, n_jobs=1,
                       random_state=1, warm_start=True)}, 3: {'model_id': 3, 'rank': 2, 'cost': 0.011235955056179803, 'ensemble_weight': 0.12, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f55d5954730>, 'balancing': Balancing(random_state=1, strategy='weighting'), 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocess

# 2nd Example: Holdout resampling

Now, let's switch to a different dataset, breast_cancer

In [ ]:
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

import autosklearn.classification

X, y = sklearn.datasets.load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = \
    sklearn.model_selection.train_test_split(X, y, random_state=1)

automl = autosklearn.classification.AutoSklearnClassifier(
          time_left_for_this_task=120,
          per_run_time_limit=30,
          disable_evaluator_output=False,
          resampling_strategy='holdout',
          resampling_strategy_arguments={'train_size': 0.80}
          )

automl.fit(X_train, y_train, dataset_name='breast_cancer')

y_hat = automl.predict(X_test)
test_acc = sklearn.metrics.accuracy_score(y_test, y_hat)
print("Test Accuracy score: {0}".format(test_acc))

[WARNING] [2019-05-14 15:53:59,451:EnsembleBuilder(1):breast_cancer] No models better than random - using Dummy Score!
[WARNING] [2019-05-14 15:53:59,463:EnsembleBuilder(1):breast_cancer] No models better than random - using Dummy Score!
[WARNING] [2019-05-14 15:55:39,649:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-05-14 15:55:39,649:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
1
['/tmp/autosklearn_tmp_707_263/.auto-sklearn/ensembles/1.0000000000.ensemble', '/tmp/autosklearn_tmp_707_263/.auto-sklearn/ensembles/1.0000000001.ensemble', '/tmp/autosklearn_tmp_707_263/.auto-sklearn/ensembles/1.0000000002.ensemble', '/tmp/autosklearn_tmp_707_263/.auto-sklearn/ensembles/1.0000000003.ensemble', '/tmp/autosklearn_tmp_707_263/.auto-sklearn/ensembles/1.0000000004.ensemble', '/tmp/autosklearn_tmp_707_263/.auto-sklearn/ensembles/1.000